In [1]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [76]:
train = pd.read_csv("trainr_mod.csv")
train=train[train.TARGET>0]
train=train.reset_index()
train.drop(columns='index',axis=1,inplace=True)
train.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA53,PCA54,PCA55,PCA56,PCA57,PCA58,PCA59,PCA60,TARGET,ID
0,4.181428,-0.910332,-0.773058,-3.198418,-0.181863,0.879686,-4.228714,-1.373065,3.324501,3.316145,...,0.317751,0.054411,-0.862100,-0.811786,1.193465,0.045853,1.358600,-0.326306,2807855.303,142882
1,-2.542522,-1.452941,0.212599,1.610610,-0.716498,-0.475858,1.334970,0.010566,0.007277,1.137324,...,0.696556,-0.224298,-0.640154,0.145018,0.565611,0.570823,0.054503,-0.693858,5522301.175,150823
2,-2.602835,-1.552959,0.196315,1.554822,-0.680184,-0.465097,1.299903,-0.023885,0.062180,1.023798,...,0.322874,-0.106931,-0.224447,-0.006227,0.167373,0.227235,-0.010886,-0.242315,1140269.958,60361
3,-2.603138,-1.547018,0.202614,1.566083,-0.681302,-0.467887,1.316297,-0.011732,0.057559,1.032979,...,0.291694,-0.098222,-0.258216,0.036754,0.193116,0.232061,0.016849,-0.269106,5520416.794,25300
4,0.111853,0.656459,-0.151232,-0.873589,0.682329,0.192093,-0.714726,0.077202,-0.266688,-0.613840,...,0.121458,0.168194,-0.074006,0.518768,0.133230,-0.886944,0.099041,0.421012,5528699.301,1576


In [88]:
test = pd.read_csv("xgbtestr_mod.csv")
test.drop(columns=['Target_Class'],axis=1,inplace=True)
print(test.shape)
test.head()

(324, 61)


,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA52,PCA53,PCA54,PCA55,PCA56,PCA57,PCA58,PCA59,PCA60,ID
0,-3.371240,-2.215935,0.563488,2.560861,-1.249154,-0.456948,2.442207,-0.065284,0.701469,2.832317,...,-0.120236,0.285423,-0.026887,-0.156723,-0.057455,0.106437,0.193890,-0.027848,-0.090868,109204
1,-3.154489,-1.946837,0.562105,2.645810,-1.282912,-0.465016,2.480607,-0.007433,0.579434,3.045714,...,-0.613556,0.887800,-0.187185,-0.502802,0.282662,0.467102,0.649275,-0.355633,-0.481405,19292
2,-2.979214,-1.760702,0.498535,2.290620,-1.031587,-0.361830,2.209101,-0.020731,0.577267,2.746204,...,-0.467445,0.347917,-0.035253,0.402488,0.402944,-0.073519,-0.293258,-0.458494,-0.121267,95061
3,30.166281,-10.466953,1.859827,34.408774,31.251488,-5.073555,-2.379895,2.491182,-4.456845,-8.015057,...,15.110787,10.257758,16.514487,2.643923,0.678703,-4.769470,9.015645,-4.066872,9.377352,119724
4,-2.616331,-1.596509,0.189329,1.517016,-0.651932,-0.452626,1.294426,-0.031824,0.094344,0.974626,...,0.010648,-0.000781,-0.010217,0.081405,-0.052867,-0.122955,-0.074151,0.026943,0.030918,88610


In [89]:
test_act=pd.read_csv("Result Evaluation.csv")
test_act=test_act[test_act.TARGET>0]
test_act=test_act.reset_index()
test_act.drop(columns='index',axis=1,inplace=True)
test_act.rename(columns={'TARGET':'Actual_Target'}, inplace=True) 
test_act.head()

,ID,Actual_Target
0,30962,4471984.113
1,91117,3798074.612
2,112309,3893559.463
3,9381,4654342.330
4,95941,2616949.737


In [90]:
#Define target and ID columns:
target = 'TARGET'
IDcol = ['ID']
from sklearn import model_selection, metrics

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=10, scoring='neg_mean_squared_error')
    cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    #print("Train CV score : ", cv_train_score)
    #print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    print("Train MAPE", np.mean(np.abs((dtrain[target].values - dtrain_predictions)/dtrain[target].values)) * 100)
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    #print("Train MAPE", np.mean(np.abs((dtest[target], dtrain_predictions)/dtrain[target].values)) * 100)
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    act=pd.merge(test_act,submission,how='left')
    #print(act)
    act.to_csv('actcomp'+filename,index=False)
    submission.to_csv(filename, index=False)

In [91]:
from sklearn.linear_model import LinearRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
#print("predictors : ",predictors)
# print predictors
alg1 = LinearRegression(normalize=False)
modelfit(alg1, train, test, predictors, target, IDcol, 'Linear.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#print(coef1)
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train MAPE 0.8250140800626169


In [92]:
from sklearn.linear_model import Ridge
predictors = [x for x in train.columns if x not in [target]+IDcol]
#print("predictors : ",predictors)
# print predictors
alg1 = Ridge(alpha=0.65, solver='auto',normalize=False)
#Ridge(alpha = 1.0,normalize=)
modelfit(alg1, train, test, predictors, target, IDcol, 'LinRidge.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train MAPE 1.383440465981345


In [93]:
from sklearn.linear_model import Lasso
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = Lasso(alpha=.5
             ,    normalize=False)
#Lasso(alpha = 1)
modelfit(alg1, train, test, predictors, target, IDcol, 'LinLaszo.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train MAPE 1.3594068417782486


In [52]:
from sklearn.linear_model import ElasticNet

In [83]:
from sklearn.linear_model import ElasticNet
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = ElasticNet(alpha=.5, l1_ratio=0.7,    normalize=False)
#Lasso(alpha = 1)
modelfit(alg1, train, test, predictors, target, IDcol, 'LinLaszo.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train MAPE 17.783006720269796
